# Make Your Own Database

The COSIMA cookbook uses a database to access information about experiments and to help with loading model output. We maintain a default database for ACCESS-OM2 experiments, but there are occasions when you might want to make your own database. This tutorial outlines the process of making your own private database.

**Requirements:** We recommend that you use `conda/analysis3-20.01` (or later) kernel on NCI (or your own up-to-date cookbook installation).

In [2]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

import cosima_cookbook as cc

**First, create a database session** using the inbuilt `create_session` function. To do this, you need to specify a path for the database - choose a location where you have write permission (that is, not the example that I have given here):

In [10]:
db = '/g/data/e14/rmh561/access-om2/archive/databases/cc_database_omip_cycle1'
session = cc.database.create_session(db,debug=True)

2020-10-25 12:23:20,479 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-10-25 12:23:20,481 INFO sqlalchemy.engine.base.Engine ()
2020-10-25 12:23:20,484 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-10-25 12:23:20,484 INFO sqlalchemy.engine.base.Engine ()
2020-10-25 12:23:20,486 INFO sqlalchemy.engine.base.Engine PRAGMA user_version
2020-10-25 12:23:20,486 INFO sqlalchemy.engine.base.Engine ()
2020-10-25 12:23:20,489 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keyword_assoc")
2020-10-25 12:23:20,490 INFO sqlalchemy.engine.base.Engine ()
2020-10-25 12:23:20,492 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("experiments")
2020-10-25 12:23:20,493 INFO sqlalchemy.engine.base.Engine ()
2020-10-25 12:23:20,494 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("keywords")
2020-10-25 12:23:20,495 INFO sqlalchemy.engine.base.Engine ()
2020-10-25 12:23:20,4

Note that you need to create the database session every time you start up your notebook; you can then update this database however many times you like.

**Now you are ready to build a database.** First, select which *experiments* you want to include in your database. For these purposes, an *experiment* is a directory containing output from a single simulation. (If you use a higher level directory you won't be able to distinguish between experiments.) 

My example below constructs a list of two experiment directories; we have chosen two cases with different resolution. The database will be built to index all netcdf files in each directory.

In [ ]:
dir_list=['/scratch/e14/rmh561/access-om2/archive/025deg_jra55_iaf_cycle1/',
          '/scratch/e14/rmh561/access-om2/archive/025deg_jra55_iaf_newIC/'
          ]

cc.database.build_index(dir_list, session, update=True)

Indexing experiment: 025deg_jra55_iaf_cycle1


100%|██████████| 23683/23683 [1:37:25<00:00,  4.05it/s]


Indexing experiment: 025deg_jra55_iaf_newIC


100%|██████████| 23064/23064 [1:35:58<00:00,  4.01it/s]  


Note that this operation may take a little while first time through, but is relatively painless to update -- **provided that you have the**  `update=True` **flag**.

Now you have your own database! 🎉

Remember to **specify** your own database when you load model output, or else it will look for your experiment in the default database.

## Using the database
To know how to effectively use this database, please see the companion tutorial: [`COSIMA_CookBook_Tutorial.ipynb`](https://cosima-recipes.readthedocs.io/en/latest/tutorials/COSIMA_CookBook_Tutorial.html#gallery-tutorials-cosima-cookbook-tutorial-ipynb). Alternatively, here is a sample that shows how you might load a variable from an experiment in your database.

In [11]:
expt = '025deg_jra55_iaf_cycle1'
variable = 'temp_global_ave'
darray = cc.querying.getvar(expt, variable, session)
annual_average = darray.resample(time='A').mean(dim='time')
annual_average.plot()

2020-10-25 12:23:31,607 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-10-25 12:23:31,610 INFO sqlalchemy.engine.base.Engine SELECT ncfiles.id AS ncfiles_id, ncfiles.index_time AS ncfiles_index_time, ncfiles.ncfile AS ncfiles_ncfile, ncfiles.present AS ncfiles_present, ncfiles.experiment_id AS ncfiles_experiment_id, ncfiles.time_start AS ncfiles_time_start, ncfiles.time_end AS ncfiles_time_end, ncfiles.frequency AS ncfiles_frequency, ncvars.id AS ncvars_id, ncvars.ncfile_id AS ncvars_ncfile_id, ncvars.variable_id AS ncvars_variable_id, ncvars.dimensions AS ncvars_dimensions, ncvars.chunking AS ncvars_chunking 
FROM ncfiles JOIN ncvars ON ncfiles.id = ncvars.ncfile_id JOIN experiments ON experiments.id = ncfiles.experiment_id 
WHERE (EXISTS (SELECT 1 
FROM variables 
WHERE variables.id = ncvars.variable_id AND variables.name = ?)) AND experiments.experiment = ? AND ncfiles.present = 1 ORDER BY ncfiles.time_start
2020-10-25 12:23:31,611 INFO sqlalchemy.engine.base.Engine ('temp

DatabaseError: (sqlite3.DatabaseError) database disk image is malformed
[SQL: SELECT ncfiles.id AS ncfiles_id, ncfiles.index_time AS ncfiles_index_time, ncfiles.ncfile AS ncfiles_ncfile, ncfiles.present AS ncfiles_present, ncfiles.experiment_id AS ncfiles_experiment_id, ncfiles.time_start AS ncfiles_time_start, ncfiles.time_end AS ncfiles_time_end, ncfiles.frequency AS ncfiles_frequency, ncvars.id AS ncvars_id, ncvars.ncfile_id AS ncvars_ncfile_id, ncvars.variable_id AS ncvars_variable_id, ncvars.dimensions AS ncvars_dimensions, ncvars.chunking AS ncvars_chunking 
FROM ncfiles JOIN ncvars ON ncfiles.id = ncvars.ncfile_id JOIN experiments ON experiments.id = ncfiles.experiment_id 
WHERE (EXISTS (SELECT 1 
FROM variables 
WHERE variables.id = ncvars.variable_id AND variables.name = ?)) AND experiments.experiment = ? AND ncfiles.present = 1 ORDER BY ncfiles.time_start]
[parameters: ('temp_global_ave', '025deg_jra55_iaf_cycle1')]
(Background on this error at: http://sqlalche.me/e/13/4xp6)

If you want to know more about the inbuilt functions used above, you can use the help function at any time, for example:

In [5]:
help(cc.database.create_session)

Help on function create_session in module cosima_cookbook.database:

create_session(db=None, debug=False)
    Create a session for the specified database file.
    
    If debug=True, the session will output raw SQL whenever it is executed on the database.



In [6]:
help(cc.querying.getvar)

Help on function getvar in module cosima_cookbook.querying:

getvar(expt, variable, session, ncfile=None, start_time=None, end_time=None, n=None, **kwargs)
    For a given experiment, return an xarray DataArray containing the
    specified variable.
    
    expt - text string indicating the name of the experiment
    variable - text string indicating the name of the variable to load
    session - a database session created by cc.database.create_session()
    ncfile - may be used if disambiguation based on filename is required
    start_time - only load data after this date. specify as a text string,
                 e.g. '1900-01-01'
    end_time - only load data before this date. specify as a text string,
               e.g. '1900-01-01'
    n - after all other queries, restrict the total number of files to the
        first n. pass a negative value to restrict to the last n
    
    Note that if start_time and/or end_time are used, the time range
    of the resulting dataset may not